In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define latent dimension for the generator input
latent_dim = 100

# Fetch historical data for the 10-year U.S. Treasury yield (interest rate proxy)
ticker = '^TNX'  # Symbol for the 10-year U.S. Treasury yield (in percentage)
data = yf.download(ticker, start='2010-01-01', end='2024-01-01')

# Use the 'Adj Close' as the interest rate (percentage)
interest_rates = data['Adj Close']

# Normalize the interest rates for GAN
interest_rates_normalized = (interest_rates - interest_rates.mean()) / interest_rates.std()
interest_rates_normalized = interest_rates_normalized.values.reshape(-1, 1)

# Build the Generator model
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='tanh'))  # Output is one interest rate value
    return model

# Build the Discriminator model
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(1,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Output probability of real or fake
    return model

# Build the GAN (combining generator and discriminator)
def build_gan(generator, discriminator):
    # Freeze discriminator when training the generator
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Compile the models
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

generator = build_generator(latent_dim)

# The GAN model combines the generator and the discriminator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Hyperparameters
batch_size = 32
epochs = 10000
half_batch = batch_size // 2


[*********************100%***********************]  1 of 1 completed
/home/j/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# ... (data loading and preprocessing as before)
# Define latent dimension for the generator input
latent_dim = 100
# Build the Generator model
def build_generator(latent_dim):
    model = models.Sequential() # The missing line!
    model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='tanh'))
    return model

# Build the Discriminator model
def build_discriminator():
    model = models.Sequential() # The missing line!
    model.add(layers.Dense(64, activation='relu', input_shape=(1,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Build the GAN (combining generator and discriminator)
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Create the discriminator and generator
discriminator = build_discriminator()
generator = build_generator(latent_dim)

# Create and compile the GAN *AFTER* freezing the discriminator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Compile the discriminator separately
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

generator = build_generator(latent_dim)

# The GAN model combines the generator and the discriminator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Hyperparameters
batch_size = 32
epochs = 10000
half_batch = batch_size // 2
for epoch in range(epochs):
  # Train discriminator with real and fake data
  idx = np.random.randint(0, interest_rates_normalized.shape[0], half_batch)
  real_rates = interest_rates_normalized[idx]

  noise = np.random.normal(0, 1, (half_batch, latent_dim))
  fake_rates = generator.predict(noise)

  # Labels for real and fake data (ensure the shape is (half_batch, 1))
  real_labels = np.ones((half_batch, 1))
  fake_labels = np.zeros((half_batch, 1))

  # Train the discriminator (real vs fake)
  d_loss_real = discriminator.train_on_batch(real_rates, real_labels)
  d_loss_fake = discriminator.train_on_batch(fake_rates, fake_labels)
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  # Train the generator via the GAN model (fooling the discriminator)
  noise = np.random.normal(0, 1, (batch_size, latent_dim))
  valid_labels = np.ones((batch_size, 1))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step


AttributeError: 'NoneType' object has no attribute 'update_state'

In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define latent dimension for the generator input
latent_dim = 100

# Fetch historical data for the 10-year U.S. Treasury yield (interest rate proxy)
ticker = '^TNX'  # Symbol for the 10-year U.S. Treasury yield (in percentage)
data = yf.download(ticker, start='2010-01-01', end='2024-01-01')

# Use the 'Adj Close' as the interest rate (percentage)
interest_rates = data['Adj Close']

# Normalize the interest rates for GAN
interest_rates_normalized = (interest_rates - interest_rates.mean()) / interest_rates.std()
interest_rates_normalized = interest_rates_normalized.values.reshape(-1, 1)

# Build the Generator model
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='tanh'))  # Output is one interest rate value
    return model

# Build the Discriminator model
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(1,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Output probability of real or fake
    return model

# Build the GAN (combining generator and discriminator)
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Create and compile the discriminator and generator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

generator = build_generator(latent_dim)

# Create and compile the GAN after freezing the discriminator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Hyperparameters
batch_size = 32
epochs = 10000
half_batch = batch_size // 2

# Training loop
for epoch in range(epochs):
    # Train discriminator with real and fake data
    idx = np.random.randint(0, interest_rates_normalized.shape[0], half_batch)
    real_rates = interest_rates_normalized[idx]

    noise = np.random.normal(0, 1, (half_batch, latent_dim))
    fake_rates = generator.predict(noise)

    # Labels for real and fake data (ensure the shape is (half_batch, 1))
    real_labels = np.ones((half_batch, 1))
    fake_labels = np.zeros((half_batch, 1))

    # Train the discriminator (real vs fake)
    d_loss_real = discriminator.train_on_batch(real_rates, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_rates, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator via the GAN model (fooling the discriminator)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))  # We want to fool the discriminator into thinking it's real
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print the progress
    if epoch % 1000 == 0:
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {d_loss[1]*100}%] [G loss: {g_loss}]")

# Generate synthetic interest rates after training
noise = np.random.normal(0, 1, (1000, latent_dim))  # Generate synthetic samples
synthetic_interest_rates = generator.predict(noise)

# Rescale back to original interest rate scale
synthetic_interest_rates_rescaled = synthetic_interest_rates * interest_rates.std() + interest_rates.mean()

# Plot real vs synthetic interest rates
plt.figure(figsize=(10, 6))
plt.plot(interest_rates[:1000], label='Real Interest Rates')
plt.plot(synthetic_interest_rates_rescaled, label='Generated Interest Rates', linestyle='dashed')
plt.legend()
plt.show()


[*********************100%***********************]  1 of 1 completed
/home/j/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


AttributeError: 'NoneType' object has no attribute 'update_state'

In [18]:
real_rates, real_labels

(array([[ 0.35251746],
        [-0.38564774],
        [ 0.49300703],
        [-0.20348774],
        [-0.7011539 ],
        [-1.8453102 ],
        [-0.02251795],
        [-1.2369192 ],
        [ 0.7501744 ],
        [ 0.09058806],
        [-0.8999824 ],
        [-0.608288  ],
        [ 0.66921425],
        [-0.56661725],
        [ 0.01081841],
        [-0.44755846]], dtype=float32),
 array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]]))

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define latent dimension for the generator input
latent_dim = 100

# Fetch historical data for the 10-year U.S. Treasury yield (interest rate proxy)
ticker = '^TNX'  # Symbol for the 10-year U.S. Treasury yield (in percentage)
data = yf.download(ticker, start='2010-01-01', end='2024-01-01')

# Use the 'Adj Close' as the interest rate (percentage)
interest_rates = data['Adj Close']

# Normalize the interest rates for GAN
interest_rates_normalized = (interest_rates - interest_rates.mean()) / interest_rates.std()
interest_rates_normalized = interest_rates_normalized.values.reshape(-1, 1)

# Build the Generator model
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='tanh'))  # Output is one interest rate value
    return model

# Build the Discriminator model
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(1,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Output probability of real or fake
    return model

# Build the GAN (combining generator and discriminator)
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Create and compile the discriminator and generator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

generator = build_generator(latent_dim)

# Create and compile the GAN after freezing the discriminator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Hyperparameters
batch_size = 32
epochs = 10000
half_batch = batch_size // 2

# Training loop
for epoch in range(epochs):
    # Train discriminator with real and fake data
    idx = np.random.randint(0, interest_rates_normalized.shape[0], half_batch)
    real_rates = interest_rates_normalized[idx]

    noise = np.random.normal(0, 1, (half_batch, latent_dim))
    fake_rates = generator.predict(noise)

    # Ensure data is correctly shaped as (half_batch, 1) and cast to float32
    real_rates = real_rates.astype(np.float32)
    fake_rates = fake_rates.astype(np.float32)

    # Labels for real and fake data (ensure the shape is (half_batch, 1))
    real_labels = np.ones((half_batch, 1))
    fake_labels = np.zeros((half_batch, 1))

    # Train the discriminator (real vs fake)
    d_loss_real = discriminator.train_on_batch(real_rates, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_rates, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator via the GAN model (fooling the discriminator)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))  # We want to fool the discriminator into thinking it's real
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print the progress
    if epoch % 1000 == 0:
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {d_loss[1]*100}%] [G loss: {g_loss}]")

# Generate synthetic interest rates after training
noise = np.random.normal(0, 1, (1000, latent_dim))  # Generate synthetic samples
synthetic_interest_rates = generator.predict(noise)

# Rescale back to original interest rate scale
synthetic_interest_rates_rescaled = synthetic_interest_rates * interest_rates.std() + interest_rates.mean()

# Plot real vs synthetic interest rates
plt.figure(figsize=(10, 6))
plt.plot(interest_rates[:1000], label='Real Interest Rates')
plt.plot(synthetic_interest_rates_rescaled, label='Generated Interest Rates', linestyle='dashed')
plt.legend()
plt.show()


[*********************100%***********************]  1 of 1 completed
/home/j/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step


AttributeError: 'NoneType' object has no attribute 'update_state'

In [20]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define latent dimension for the generator input
latent_dim = 100

# Fetch historical data for the 10-year U.S. Treasury yield (interest rate proxy)
ticker = '^TNX'  # Symbol for the 10-year U.S. Treasury yield (in percentage)
data = yf.download(ticker, start='2010-01-01', end='2024-01-01')

# Use the 'Adj Close' as the interest rate (percentage)
interest_rates = data['Adj Close']

# Normalize the interest rates for GAN
interest_rates_normalized = (interest_rates - interest_rates.mean()) / interest_rates.std()
interest_rates_normalized = interest_rates_normalized.values.reshape(-1, 1)

# Build the Generator model
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='tanh'))  # Output is one interest rate value
    return model

# Build the Discriminator model
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(1,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Output probability of real or fake
    return model

# Build the GAN (combining generator and discriminator)
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Create and compile the discriminator and generator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

generator = build_generator(latent_dim)

# Create and compile the GAN after freezing the discriminator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Hyperparameters
batch_size = 32
epochs = 10000
half_batch = batch_size // 2

# Training loop
for epoch in range(epochs):
    # Train discriminator with real and fake data
    idx = np.random.randint(0, interest_rates_normalized.shape[0], half_batch)
    real_rates = interest_rates_normalized[idx]

    noise = np.random.normal(0, 1, (half_batch, latent_dim))
    fake_rates = generator.predict(noise)

    # Ensure data is correctly shaped as (half_batch, 1) and cast to float32
    real_rates = real_rates.astype(np.float32)
    fake_rates = fake_rates.astype(np.float32)

    # Labels for real and fake data (ensure the shape is (half_batch, 1))
    real_labels = np.ones((half_batch, 1))
    fake_labels = np.zeros((half_batch, 1))

    # Train the discriminator (real vs fake)
    d_loss_real = discriminator.train_on_batch(real_rates, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_rates, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator via the GAN model (fooling the discriminator)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))  # We want to fool the discriminator into thinking it's real
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print the progress
    if epoch % 1000 == 0:
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {d_loss[1]*100}%] [G loss: {g_loss}]")

# Generate synthetic interest rates after training
noise = np.random.normal(0, 1, (1000, latent_dim))  # Generate synthetic samples
synthetic_interest_rates = generator.predict(noise)

# Rescale back to original interest rate scale
synthetic_interest_rates_rescaled = synthetic_interest_rates * interest_rates.std() + interest_rates.mean()

# Plot real vs synthetic interest rates
plt.figure(figsize=(10, 6))
plt.plot(interest_rates[:1000], label='Real Interest Rates')
plt.plot(synthetic_interest_rates_rescaled, label='Generated Interest Rates', linestyle='dashed')
plt.legend()
plt.show()


[*********************100%***********************]  1 of 1 completed
/home/j/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


AttributeError: 'NoneType' object has no attribute 'update_state'

In [22]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

# Define latent dimension for the generator input
latent_dim = 100

# Create a simple generator model
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='tanh'))  # Output is a single value (e.g., a random number)
    return model

# Create a simple discriminator model
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(1,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Output probability of real or fake
    return model

# Combine generator and discriminator to create the GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze discriminator weights during GAN training
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Initialize the models
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

generator = build_generator(latent_dim)

# Initialize the GAN
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Hyperparameters
batch_size = 32
epochs = 10000
half_batch = batch_size // 2

# Training loop
for epoch in range(epochs):
    # Generate random data for real and fake
    real_data = np.random.normal(0, 1, (half_batch, 1))  # Random "real" data (for simplicity)
    noise = np.random.normal(0, 1, (half_batch, latent_dim))  # Random noise for the generator
    fake_data = generator.predict(noise)  # Generate fake data from noise

    # Labels for real and fake data
    real_labels = np.ones((half_batch, 1))  # Real data should be labeled as 1
    fake_labels = np.zeros((half_batch, 1))  # Fake data should be labeled as 0

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)  # Combine losses

    # Train the generator (via the GAN)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))  # We want to fool the discriminator into thinking it's real
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print progress every 1000 epochs
    if epoch % 1000 == 0:
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {d_loss[1]*100}%] [G loss: {g_loss}]")

# Generate synthetic data after training
noise = np.random.normal(0, 1, (1000, latent_dim))  # Generate synthetic samples
synthetic_data = generator.predict(noise)

# Rescale back to the range of real data
synthetic_data_rescaled = synthetic_data * 2  # Adjust as needed based on your data range

# Plot real vs synthetic data
plt.figure(figsize=(10, 6))
plt.plot(np.random.normal(0, 1, (1000, 1)), label='Real Data')
plt.plot(synthetic_data_rescaled, label='Generated Data', linestyle='dashed')
plt.legend()
plt.show()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


AttributeError: 'NoneType' object has no attribute 'update_state'

In [28]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

# Set the latent dimension for the generator input
latent_dim = 100

# Build the Generator model
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='tanh'))  # Output is a single value (e.g., a number)
    return model

# Build the Discriminator model
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(1,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Output probability of real or fake
    return model

# Build the GAN (combining the generator and the discriminator)
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze discriminator weights during GAN training
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Initialize the models
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

generator = build_generator(latent_dim)

# Create the GAN by combining the generator and discriminator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Hyperparameters
batch_size = 32
epochs = 10000
half_batch = batch_size // 2

# Training loop
for epoch in range(epochs):
    # Generate random data for real (normal distribution) and fake (from generator)
    real_data = np.random.normal(0, 1, (half_batch, 1))  # Random "real" data (for simplicity)
    noise = np.random.normal(0, 1, (half_batch, latent_dim))  # Random noise for the generator
    fake_data = generator.predict(noise)  # Generate fake data from noise

    # Labels for real and fake data
    real_labels = np.ones((half_batch, 1))  # Real data labeled as 1
    fake_labels = np.zeros((half_batch, 1))  # Fake data labeled as 0

    # Train the discriminator with real and fake data
    print(real_data, real_labels,epoch, "real_data, real_labels")
    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)  # Combine losses

    # Train the generator via the GAN (the goal is to fool the discriminator into thinking fake data is real)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))  # Generate random noise
    valid_labels = np.ones((batch_size, 1))  # We want to fool the discriminator into thinking it's real
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print progress every 1000 epochs
    if epoch % 1000 == 0:
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {d_loss[1]*100}%] [G loss: {g_loss}]")

# Generate synthetic data after training
noise = np.random.normal(0, 1, (1000, latent_dim))  # Generate synthetic samples
synthetic_data = generator.predict(noise)

# Rescale back to the range of real data (adjust as needed based on your data range)
synthetic_data_rescaled = synthetic_data * 2  # Adjust as needed based on your data range

# Plot real vs synthetic data
plt.figure(figsize=(10, 6))
plt.plot(np.random.normal(0, 1, (1000, 1)), label='Real Data')
plt.plot(synthetic_data_rescaled, label='Generated Data', linestyle='dashed')
plt.legend()
plt.show()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
[[-1.02157988e+00]
 [ 2.17732132e-01]
 [-1.00906496e+00]
 [-2.22046180e-01]
 [ 3.30366692e-01]
 [-4.16453774e-01]
 [ 4.77158219e-04]
 [ 2.06486265e-02]
 [-7.68665628e-01]
 [-6.94755965e-01]
 [ 1.21195233e+00]
 [-1.09938395e+00]
 [ 5.31690486e-01]
 [ 7.08998805e-01]
 [ 9.61205808e-02]
 [ 8.13287469e-01]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]] 0 real_data, real_labels


AttributeError: 'NoneType' object has no attribute 'update_state'